In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
pages = range(2,25)
url = f"https://twitchtracker.com/channels/viewership?page=1"
def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

In [5]:
url = "https://twitchtracker.com/channels/viewership?page="
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

df = get_data(url, 25)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [9]:
def streamers_df_cleaning(df):
    df = df[df["Time Streamed"].str.contains("ads")==False]
    # creating a new column that contains hours where we replace all hours with an emptry string and convert numbers in string to float
    df['Time Streamed (in hours)'] = df['Time Streamed'].str.replace('hours','').apply(lambda x: float(x))


    # do same thing for hours watched
    df['Hours Watched'] = df['Hours Watched'].apply(lambda x: int(x))

    # do it for every other columns
    df['AVG Viewers'] = df['AVG Viewers'].apply(lambda x: int(x))
    df['All Time Peak Viewers'] = df['All Time Peak Viewers'].apply(lambda x: int(x))
    df['Total Followers'] = df['Total Followers'].apply(lambda x: int(x))
    df['Total Views'] = df['Total Views'].apply(lambda x: int(x))
    df['Followers Gained'] = df['Followers Gained'].str.replace('+','').apply(lambda x: int(x))
    # check if it worked
    df.dtypes

    # drop and rename columns
    df.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Time Streamed'], axis=1,inplace=True)
    df.rename(columns={'Unnamed: 2':'username'}, inplace=True)

    # sort according to importance
    df = df[['username', 'AVG Viewers', 'Time Streamed (in hours)', 'Hours Watched', 'Followers Gained', 'Total Followers', 'Total Views', 'All Time Peak Viewers']]
    return df

In [10]:
clean_streamers_df = streames_df_cleaning(df)

NameError: name 'streames_df_cleaning' is not defined

In [ ]:
#get df for every broadcast //done

def parse_games_2(user_name):
    user_name.lower()
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url = f"https://twitchtracker.com/{user_name}/games", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    return df

In [ ]:
quack=parse_games_2('Quackity')

In [ ]:
quack.head()

In [ ]:
def clean_gamer_df(username):
    df=parse_games_2(username)
    df["Total airtime"] = df["Total airtime"].astype(str).str[:-1]
    df[['Followers per hour', 'to_remove']]=df['Followers'].str.split(' /',expand=True)
    df = df.drop(columns=['Followers', 'to_remove'])
    df[['Total airtime','Percentage of total airtime']]=df['Total airtime'].str.split(' ',expand=True) 
    df['Percentage of total airtime'] = df['Percentage of total airtime'].apply(lambda x: float(x))
    df['Followers per hour'] = df['Followers per hour'].apply(lambda x: float(x))
    df['Last seen'] = pd.to_datetime(df['Last seen'])
    df=df.rename(columns={'#':'id'})
    df['id']=df['id'].astype(str).str[1]
    df=df.set_index('id')
    return df

In [ ]:
clean_gamer_df('Quackity')